# Unicode strings

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Ali-Alameer/NLP/blob/main/unicode_strings.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Introduction

NLP models often handle different languages with different character sets.  *Unicode* is a standard encoding system that is used to represent characters from almost all languages.  Every Unicode character is encoded using a unique integer [code point](https://en.wikipedia.org/wiki/Code_point) between `0` and `0x10FFFF`. A *Unicode string* is a sequence of zero or more code points.

This tutorial shows how to represent Unicode strings in TensorFlow and manipulate them using Unicode equivalents of standard string ops. It separates Unicode strings into tokens based on script detection.

In [ ]:
import tensorflow as tf
import numpy as np

## The `tf.string` data type

The basic TensorFlow `tf.string` `dtype` allows you to build tensors of byte strings.
Unicode strings are [utf-8](https://en.wikipedia.org/wiki/UTF-8) encoded by default.

In [ ]:
tf.constant(u"Thanks 😊") # unicode strings are indicated by the "u" prefix.

A `tf.string` tensor treats byte strings as atomic units. This enables it to store byte strings of varying lengths. The string length is not included in the tensor dimensions.


In [ ]:
tf.constant([u"You're", u"welcome!"]).shape

If you use Python to construct strings, note that [string literals](https://docs.python.org/3/reference/lexical_analysis.html) are Unicode-encoded by default.

## Representing Unicode

There are two standard ways to represent a Unicode string in TensorFlow:

* `string` scalar — where the sequence of code points is encoded using a known [character encoding](https://en.wikipedia.org/wiki/Character_encoding).
* `int32` vector — where each position contains a single code point.

For example, the following three values all represent the Unicode string `"语言处理"` (which means "language processing" in Chinese):

In [ ]:
# Unicode string, represented as a UTF-8 encoded string scalar.
text_utf8 = tf.constant(u"语言处理")
text_utf8

In [ ]:
# Unicode string, represented as a UTF-16-BE encoded string scalar.
text_utf16be = tf.constant(u"语言处理".encode("UTF-16-BE"))
text_utf16be

In [ ]:
# Unicode string, represented as a vector of Unicode code points.
text_chars = tf.constant([ord(char) for char in u"语言处理"])
text_chars

### Converting between representations

TensorFlow provides operations to convert between these different representations:

* `tf.strings.unicode_decode`: Converts an encoded string scalar to a vector of code points.
* `tf.strings.unicode_encode`: Converts a vector of code points to an encoded string scalar.
* `tf.strings.unicode_transcode`: Converts an encoded string scalar to a different encoding.

In [ ]:
tf.strings.unicode_decode(text_utf8,
                          input_encoding='UTF-8')

In [ ]:
tf.strings.unicode_encode(text_chars,
                          output_encoding='UTF-8')

In [ ]:
tf.strings.unicode_transcode(text_utf8,
                             input_encoding='UTF8',
                             output_encoding='UTF-16-BE')

### Batch dimensions

When decoding multiple strings, the number of characters in each string may not be equal.  The return result is a [`tf.RaggedTensor`](../../guide/ragged_tensor.ipynb), where the innermost dimension length varies depending on the number of characters in each string.

In [ ]:
# A batch of Unicode strings, each represented as a UTF8-encoded string.
batch_utf8 = [s.encode('UTF-8') for s in # Encode str to UTF-8 bytes: s.encode('utf-8')
              [u'hÃllo', u'What is the weather tomorrow', u'Göödnight', u'😊']]
batch_chars_ragged = tf.strings.unicode_decode(batch_utf8,
                                               input_encoding='UTF-8')
for sentence_chars in batch_chars_ragged.to_list():
  print(sentence_chars)

You can use this `tf.RaggedTensor` directly, or convert it to a dense `tf.Tensor` with padding or a `tf.sparse.SparseTensor` using the methods `tf.RaggedTensor.to_tensor` and `tf.RaggedTensor.to_sparse`.

In [ ]:
batch_chars_padded = batch_chars_ragged.to_tensor(default_value=-1)
print(batch_chars_padded.numpy())

In [ ]:
batch_chars_sparse = batch_chars_ragged.to_sparse()
# the code below extract each of the SparseTensor elements and print them in a padded "_" style for visualisation (nurdy way but worth understanding the way its implemented)
nrows, ncols = batch_chars_sparse.dense_shape.numpy() # get the shape of the sparse tensor and convert shape to numpy array
elements = [['_' for i in range(ncols)] for j in range(nrows)] # the elements is a list with values of "_" with size 4 x 28
for (row, col), value in zip(batch_chars_sparse.indices.numpy(), batch_chars_sparse.values.numpy()):
  elements[row][col] = str(value)
# max_width = max(len(value) for row in elements for value in row)
value_lengths = []
for row in elements:
  for value in row:
    value_lengths.append(len(value))
max_width = max(value_lengths)
print('[%s]' % '\n '.join( # The %s token allows to insert a string. Notice that the %s token is replaced by whatever we pass to the string after the % symbol. "\n" indicate end of line
    '[%s]' % ', '.join(value.rjust(max_width) for value in row) # .rjust returns the string right justified in a string of length width
    for row in elements))

details about the zip function: The zip() function returns a zip object, which is an iterator of tuples where the first item in each passed iterator is paired together, and then the second item in each passed iterator are paired together etc.

If the passed iterators have different lengths, the iterator with the least items decides the length of the new iterator.

a = ("John", "Charles", "Mike")
b = ("Jenny", "Christy", "Monica")

x = zip(a, b)

#use the tuple() function to display a readable version of the result:

print(tuple(x))
(('John', 'Jenny'), ('Charles', 'Christy'), ('Mike', 'Monica'))

When encoding multiple strings with the same lengths, use a `tf.Tensor` as the input.

In [ ]:
tf.strings.unicode_encode([[99, 97, 116], [100, 111, 103], [99, 111, 119]],
                          output_encoding='UTF-8')

When encoding multiple strings with varying length, use a `tf.RaggedTensor` as the input.

In [ ]:
tf.strings.unicode_encode(batch_chars_ragged, output_encoding='UTF-8')

In [ ]:
batch_chars_ragged # input is multiple strings with varying length

If you have a tensor with multiple strings in padded or sparse format, convert it first into a `tf.RaggedTensor` before calling `tf.strings.unicode_encode`.

In [ ]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_sparse(batch_chars_sparse),
    output_encoding='UTF-8')

In [ ]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_tensor(batch_chars_padded, padding=-1),
    output_encoding='UTF-8')

## Unicode operations

### Character length

Use the `unit` parameter of the `tf.strings.length` op to indicate how character lengths should be computed.  `unit` defaults to `"BYTE"`, but it can be set to other values, such as `"UTF8_CHAR"` or `"UTF16_CHAR"`, to determine the number of Unicode codepoints in each encoded string.

In [ ]:
# Note that the final character takes up 4 bytes in UTF8.
thanks = u'Thanks 😊'.encode('UTF-8') #  unicode strings are indicated by the "u" prefix.  UTF-8 is a variable-width character encoding
num_bytes = tf.strings.length(thanks).numpy()
num_chars = tf.strings.length(thanks, unit='UTF8_CHAR').numpy()
print('{} bytes; {} UTF-8 characters'.format(num_bytes, num_chars))

### Character substrings

The `tf.strings.substr` op accepts the `unit` parameter, and uses it to determine what kind of offsets the `pos` and `len` paremeters contain.

For each string in the input Tensor, creates a substring starting at index pos with a total length of len.

If len defines a substring that would extend beyond the length of the input string, or if len is negative, then as many characters as possible are used.

A negative pos indicates distance within the string backwards from the end.

If pos specifies an index which is out of range for any of the input strings, then an InvalidArgumentError is thrown.

pos and len must have the same shape, otherwise a ValueError is thrown on Op creation.

input = [b'Hello', b'World']
position = 1
length = 3

output = [b'ell', b'orl']

In [ ]:
# Here, unit='BYTE' (default). Returns a single byte with len=1
tf.strings.substr(thanks, pos=7, len=1).numpy()

In [ ]:
# Specifying unit='UTF8_CHAR', returns a single 4 byte character in this case
print(tf.strings.substr(thanks, pos=7, len=1, unit='UTF8_CHAR').numpy())

### Split Unicode strings

The `tf.strings.unicode_split` operation splits unicode strings into substrings of individual characters.

In [ ]:
tf.strings.unicode_split(thanks, 'UTF-8').numpy()

### Byte offsets for characters

To align the character tensor generated by `tf.strings.unicode_decode` with the original string, it's useful to know the offset for where each character begins.  The method `tf.strings.unicode_decode_with_offsets` is similar to `unicode_decode`, except that it returns a second tensor containing the start offset of each character.

In [ ]:
codepoints, offsets = tf.strings.unicode_decode_with_offsets(u'🎈🎉🎊', 'UTF-8')

for (codepoint, offset) in zip(codepoints.numpy(), offsets.numpy()):
  print('At byte offset {}: codepoint {}'.format(offset, codepoint))

try different combinations of characters for the above and check the results 

## Unicode scripts

Each Unicode code point belongs to a single collection of codepoints known as a [script](https://en.wikipedia.org/wiki/Script_%28Unicode%29) .  A character's script is helpful in determining which language the character might be in. For example, knowing that 'Б' is in Cyrillic script indicates that modern text containing that character is likely from a Slavic language such as Russian or Ukrainian.

TensorFlow provides the `tf.strings.unicode_script` operation to determine which script a given codepoint uses. The script codes are `int32` values corresponding to [International Components for
Unicode](http://site.icu-project.org/home) (ICU) [`UScriptCode`](http://icu-project.org/apiref/icu4c/uscript_8h.html) values.


In [ ]:
uscript = tf.strings.unicode_script([33464, 1041])  # ['芸', 'Б']

print(uscript.numpy())  # [17, 8] == [USCRIPT_HAN, USCRIPT_CYRILLIC]

The `tf.strings.unicode_script` operation can also be applied to multidimensional `tf.Tensor`s or `tf.RaggedTensor`s of codepoints:

In [ ]:
print(tf.strings.unicode_script(batch_chars_ragged))